In [22]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from IPython.display import clear_output
import numpy as np
import keyboard
import serial
import struct
import math
import time

serialPort = serial.Serial(port = "COM4",baudrate=115200, bytesize=8,stopbits=serial.STOPBITS_ONE)
Theta = np.array([90, 134, 5, 179,  0])
T1 = Theta[2] # most important factor to change.
T2 = Theta[3]
T3 = Theta[4] # This cannot be much larger, only accurate measures must be taken
Time = Theta*0.01031 + 0.544
RB = abs(Time*24e3/16)
RB_array = np.array(RB, dtype = np.single) # this converts the array values to float values.
RB_binary = RB_array.tobytes()
for i in np.arange(1,6,1):
    voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
    serialPort.write(voyager.data)  

In [138]:
#---------------------------------------------------MONO-VISION--------------------------------------
from numpy import cos,sin,tan,degrees,radians
import numpy as np
import time
l1 = 0.13
l2 = 0.13
l3 = 0.18
lc = 0.065 # must be validated assumed 0.04
D = 0.07
hc = 0.05

#-------------------------------------------------Theta------------------------------------------
#all angles are relative to ground
T1 = 130
T2 = 42.62
T3 = -44#-40
Tc = 90 - abs(T3)
FOV = 37.04
#FOV = 35

Xc = -l1*sin(radians(T1)) + l2*cos(radians(T2)) + lc*cos(radians(T3)) + hc*cos(radians(Tc))
Yc = D + l1*cos(radians(180 - T1)) + l2*sin(radians(T2)) + lc*sin(radians(T3)) + hc*sin(radians(Tc))
print('Xcamera',Xc)
print('Ycamera',Yc)
Xp1 = Xc - Yc/tan(radians(180 - abs(T3) - FOV/2)) 
Xp2 = Xc + Yc/tan(radians(abs(Tc) - FOV/2)) 
camera_length = np.round((Xp2-Xp1),2)

print('The first pixel detected will be located at the distance of ',np.round(Xp1,3),'m')
print('The last pixel detected will be located at the distance of ',np.round(Xp2,3),'m')
print('The total distance between pixel edges are:',camera_length,'m')
fcoordinates = open("C:/ODA/models/research/object_detection/Coordinates.txt","r")
coordinates = fcoordinates.readlines()
while coordinates == []:
    fcoordinates = open("C:/ODA/models/research/object_detection/Coordinates.txt","r")
    coordinates = fcoordinates.readlines()
i = 0
value = []
for k in coordinates:
    i = i + 1
    if (i == 1) or (i == 3):
        value.append(np.round((float(k)),4))
    else:
        value.append(np.round((float(k)),4)) 
time.sleep(0.25)        
yminr = value[0]
xminr = value[1]
ymaxr = value[2]
xmaxr = value[3]

print('This is the distance to the camera from the base: ',Xc,'cm')
print('ratio value = ',ymaxr)
distance = [np.round((1-ymaxr) * camera_length + Xp1, 5) ] 
print('\nThe object is measured to be ',distance[0],'m away')

Z = distance[0]
Y = 0.03
X = 0




Xcamera 0.07756612777858013
Ycamera 0.2324038589197619
The first pixel detected will be located at the distance of  0.198 m
The last pixel detected will be located at the distance of  0.524 m
The total distance between pixel edges are: 0.33 m
This is the distance to the camera from the base:  0.07756612777858013 cm
ratio value =  0.7315

The object is measured to be  0.28705 m away


In [ ]:
import serial
serialPort = serial.Serial(port = "COM4",baudrate=115200, bytesize=8,stopbits=serial.STOPBITS_ONE)

In [126]:
#---------------------------------------Inverse Kinematics Project Voyager--------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import sys
from IPython.display import clear_output
import numpy as np
import keyboard
import serial
import struct
import math
import time
def Inverse_Kinematics(X,Y,Z,phi):
    phi = np.radians(phi)#phi = np.arccos((-l2 - l3 + actuallength)/l4)
    Offset2 = 0
    l2 = 13
    l3 = 13
    l4 = 19
    d = 3
    Theta2_list = []
    Theta3_list = []
    error_net = []
    xo = []
    yo = []
    zo = []
    actuallength = (X**2 + Y**2 + Z**2)**0.5 #maxlength = l2+l3+l4*np.cos(phi)
    MAX = l2 + l3 + l4
    Ymax = l2 + l3
    if MAX < actuallength:
        print('The following request is impossible to satisfy')
        sys.exit()  
    Theta1 = np.arctan2(Z,X)
    for i in np.arange(30,151,1):
        Theta2 = np.radians(i)  
        C3 = X/np.cos(Theta1) - l4*np.cos(phi) - l2*np.cos(Theta2)
        S3 = Y - l4*np.sin(phi) - l2*np.sin(Theta2) - d
        Theta3 = np.arctan2(S3, C3)
        Theta2_list.append(Theta2)
        Theta3_list.append(Theta3)
        Xo = np.cos(Theta1)*(l2 * np.cos(Theta2) + l3 * np.cos(Theta3) + l4 * np.cos(phi))
        Yo = d + l2 * np.sin(Theta2) + l3 * np.sin(Theta3) + l4 * np.sin(phi)
        Zo = np.sin(Theta1)*(l2 * np.cos(Theta2) + l3 * np.cos(Theta3) + l4 * np.cos(phi))
        errorX = X - Xo
        errorY = Y - Yo
        errorZ = Z - Zo
        xo.append(Xo)
        yo.append(Yo)
        zo.append(Zo)
        error_net.append((errorX**2 + errorY**2 + errorZ**2)**(1/2))    
    min_index = error_net.index(min(error_net))
    error = np.round(error_net[min_index],4)
    Theta2_optimised = Theta2_list[min_index]
    Theta3_optimised = Theta3_list[min_index]
    Xe = xo[min_index]
    Ye = yo[min_index]
    Ze = zo[min_index]
    Theta = np.round([np.degrees(Theta1), np.degrees(Theta2_optimised), np.degrees(Theta3_optimised), np.degrees(phi)], 2)
    Theta2f = Theta[1]
    Theta3f = Theta[2]
    Theta4f = Theta[3]
    Theta2m = Theta2f - Offset2
    Theta3m = 90 + Theta3f - Theta2f
    Theta4m = 90 + Theta3f - Theta4f
    Theta_motors = np.round([Theta[0], Theta2m, Theta3m, Theta4m, 0])
    return Theta_motors, error 

#---------------------------------------------------Selecting Values------------------------------------------------------
Xe = -15
Ye = 30
Ze = 15
#---------------------------------------------iterating for optimal phi--------------------------------------------------
e_list = []
T1 = []
T2 = []
T3 = []
T4 = []
T5 = []
xi = []
for phi in np.arange(-180,181,1):
    if Xe == 0:
        Tim,ei = Inverse_Kinematics(Ze,Ye,Xe,phi)
        Tim[0] = 93
    else:
        Tim,ei = Inverse_Kinematics(Xe,Ye,Ze,phi)
    Possible = True
    for k in np.arange(0,len(Tim),1):
        if (Tim[k] < 0) or (Tim[k] > 180):
            Possible = False
    if Possible:
        e_list.append(ei)
        T1.append(Tim[0])
        T2.append(Tim[1])
        T3.append(Tim[2])
        T4.append(Tim[3])
        T5.append(Tim[4])
        xi.append(phi)
if e_list == []:
    print('This cooridnate cannot be reached')
    sys.exit()
min_error_index = e_list.index(min(e_list))
error_motor = np.round(e_list[min_error_index],4)
phi_final = xi[min_error_index]
Theta_final_choice = np.round([T1[min_error_index], T2[min_error_index], T3[min_error_index], T4[min_error_index], 0])
print('The Theta final choice for the motors movement which is possible and returns the lowest error is:', Theta_final_choice)
print('The minimum error found was found to be = ',error_motor)
print('The final optimised phi position chosen is ',phi_final,' degrees')
#----------------------------------------------Sending motors their instructions-------------------------------------------                  
Theta = np.array(Theta_final_choice)
Time = Theta*0.01031 + 0.544
RB = abs(Time*24e3/16)
RB_array = np.array(RB, dtype = np.single) # this converts the array values to float values.
RB_binary = RB_array.tobytes()
for i in np.arange(1,6,1):
    voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
    serialPort.write(voyager.data)       


The Theta final choice for the motors movement which is possible and returns the lowest error is: [135. 109.  39. 137.   0.]
The minimum error found was found to be =  0.0001
The final optimised phi position chosen is  11  degrees


In [ ]:
# old version of monovision

                from numpy import cos,sin,tan,degrees,radians
                import numpy as np
                l1 = 0.13 
                l2 = 0.13
                l3 = 0.15
                lc = 0.04 # must be validated assumed 0.04
                D = 0.06
                hc = 0.05
                T1 = 30 #(Theta[1] + 35) - 90  # most important factor to change.
                T2 = 20
                T3 = 60 # This cannot be much larger, only accurate measures must be taken
                Tc = 90 - T3
                FOV = 40.1
                Xc = -l1*sin(radians(T1)) + l2*cos(radians(T2)) + lc*cos(radians(T3)) + hc*cos(radians(Tc))
                Yc = D + l1*cos(radians(T1)) + l2*sin(radians(T2)) + lc*sin(radians(T3)) + hc*sin(radians(Tc))
                Xp1 = Yc*tan(radians(Tc - FOV/2)) + Xc
                Xp2 = Yc*tan(radians(Tc + FOV/2)) + Xc
                camera_length = np.round((Xp2-Xp1)*100,2)
                i = 0
                value = []
                for k in coordinates:
                    i = i + 1
                    if (i == 1) or (i == 3):
                        value.append(np.round((float(k)),2))
                    else:
                        value.append(np.round((float(k)),2))   
                yminr = value[0]
                xminr = value[1]
                ymaxr = value[2]
                xmaxr = value[3]
                distance = [np.round((1-ymaxr) * camera_length + Xp1*100, 2)] 